In [1]:
# 패키지 불러오기
# ===== 기본 라이브러리 =====
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
import dask.dataframe as dd
from multiprocessing import Pool, cpu_count
from math import ceil

# ===== 시각화 =====
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

FONT_PATH = r"C:/Windows/Fonts/malgun.ttf"
FONT = fm.FontProperties(fname=FONT_PATH)


# ===== 출력 옵션 =====
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None) # 50
pd.set_option("display.max_colwidth", None)  # 컬럼 내용도 잘리지 않게
pd.set_option("display.width", 1000)

# ===== 시각화 스타일 =====
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

In [5]:
path = 'W:/공용 드라이브/01. 매출/00. Raw/Daily/'

# 예약 파일 리스트 (6~12월)
reserve_files = [
    f for f in os.listdir(path)
    if (
        f.startswith('예약매출_2025') and
        f.endswith('.xlsx') and
        f.count('_') == 1 and
        '20250601' <= f.split('_')[1].replace('.xlsx','') <= '20251231'
    )
]

reserve_files.sort()  # 날짜순 정렬 (권장)

dfs = []

print(f'총 파일 수: {len(reserve_files)}')

for i, file in enumerate(reserve_files, start=1):
    print(f'[{i}/{len(reserve_files)}] 로딩 중: {file}')

    df = pd.read_excel(
        os.path.join(path, file),
        index_col=0
    )

    dfs.append(df)

# 전체 concat
reserve_df = pd.concat(dfs, ignore_index=True)

# datetime 지정
reserve_df['수령일자'] = pd.to_datetime(reserve_df['수령일자'],errors='coerce')
reserve_df['취소일자'] = pd.to_datetime(reserve_df['취소일자'],errors='coerce')
reserve_df['반품일자'] = pd.to_datetime(reserve_df['반품일자'],errors='coerce')


# parquet 저장
save_path = r'C:/Users/user04/code/reserve_stock/reserve/'
os.makedirs(save_path, exist_ok=True)

reserve_df.to_parquet(
    os.path.join(save_path, 'reserve(2506_2512).parquet'),
    engine='pyarrow',
    index=False
)

print('✅ 예약 데이터 적재 완료')


총 파일 수: 212
[1/212] 로딩 중: 예약매출_20250601.xlsx
[2/212] 로딩 중: 예약매출_20250602.xlsx
[3/212] 로딩 중: 예약매출_20250603.xlsx
[4/212] 로딩 중: 예약매출_20250604.xlsx
[5/212] 로딩 중: 예약매출_20250605.xlsx
[6/212] 로딩 중: 예약매출_20250606.xlsx
[7/212] 로딩 중: 예약매출_20250607.xlsx
[8/212] 로딩 중: 예약매출_20250608.xlsx
[9/212] 로딩 중: 예약매출_20250609.xlsx
[10/212] 로딩 중: 예약매출_20250610.xlsx
[11/212] 로딩 중: 예약매출_20250611.xlsx
[12/212] 로딩 중: 예약매출_20250612.xlsx
[13/212] 로딩 중: 예약매출_20250613.xlsx
[14/212] 로딩 중: 예약매출_20250615.xlsx
[15/212] 로딩 중: 예약매출_20250616.xlsx
[16/212] 로딩 중: 예약매출_20250617.xlsx
[17/212] 로딩 중: 예약매출_20250618.xlsx
[18/212] 로딩 중: 예약매출_20250619.xlsx
[19/212] 로딩 중: 예약매출_20250620.xlsx
[20/212] 로딩 중: 예약매출_20250621.xlsx
[21/212] 로딩 중: 예약매출_20250622.xlsx
[22/212] 로딩 중: 예약매출_20250623.xlsx
[23/212] 로딩 중: 예약매출_20250624.xlsx
[24/212] 로딩 중: 예약매출_20250625.xlsx
[25/212] 로딩 중: 예약매출_20250626.xlsx
[26/212] 로딩 중: 예약매출_20250627.xlsx
[27/212] 로딩 중: 예약매출_20250628.xlsx
[28/212] 로딩 중: 예약매출_20250629.xlsx
[29/212] 로딩 중: 예약매출_20250630.xlsx
[30/212] 로딩

C:\Users\user04\AppData\Local\Temp\ipykernel_2572\1132113736.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reserve_df['수령일자'] = pd.to_datetime(reserve_df['수령일자'],errors='coerce')
C:\Users\user04\AppData\Local\Temp\ipykernel_2572\1132113736.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reserve_df['취소일자'] = pd.to_datetime(reserve_df['취소일자'],errors='coerce')
C:\Users\user04\AppData\Local\Temp\ipykernel_2572\1132113736.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reserve_df['반품일자'] = pd.to_datetime(reserve_df['반품일자'],errors='coerce')


In [7]:
import pandas as pd
import os

path = 'W:/공용 드라이브/02. 재고/01. 재고현황 (일자별)/2025/'
save_path = r'C:/Users/user04/code/reserve_stock/reserve/base_stock_parts/'
os.makedirs(save_path, exist_ok=True)

target_files.sort(key=lambda x: x[1])  # 날짜순 정렬

for i, (file_path, date) in enumerate(target_files, start=1):
    part_file = os.path.join(save_path, f'base_stock_{date}.parquet')

    # 이미 처리한 날짜면 스킵
    if os.path.exists(part_file):
        print(f'⏩ [{i}] {date}: 이미 저장됨 → 건너뜀')
        continue

    try:
        print(f'[{i}] {date}: 로딩 중 → {file_path}')

        if os.path.getsize(file_path) == 0:
            print(f'⚠️ 빈 파일 → 건너뜀')
            continue

        df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

        df.columns = (
            df.columns
            .astype(str)
            .str.strip()
            .str.replace(' ', '', regex=False)
        )

        df['기준일자'] = date

        # 날짜별 parquet 저장
        df.to_parquet(part_file, engine='pyarrow', index=False)
        print(f'✔ 저장 완료: {part_file}')

    except pd.errors.EmptyDataError:
        print(f'⚠️ EmptyDataError → 건너뜀: {file_path}')
    except Exception as e:
        print(f'❌ 예외 발생 → {file_path} → {e}')


In [10]:
import duckdb

duckdb.query(f"""
    COPY (
        SELECT * FROM parquet_scan('{save_path}/base_stock_*.parquet', union_by_name=True)
    )
    TO '{save_path}/../base_stock(2506_2512).parquet'
    (FORMAT 'parquet');
""")

print("✅ DuckDB로 병합 및 저장 완료!")

✅ DuckDB로 병합 및 저장 완료!


In [ ]:
import dask.dataframe as dd
import os

# 모든 파일을 lazy하게 로드
df_all = dd.read_parquet(
    os.path.join(save_path, 'base_stock_*.parquet'),
    engine='pyarrow'
)

# 병합 + 저장
df_all.to_parquet(
    os.path.join(save_path, 'base_stock(2506_2512).parquet'),
    engine='pyarrow',
    write_index=False  # 인덱스 저장 방지
)

print("✅ 전체 병합 및 저장 완료 (Dask 사용)")

In [ ]:
# =========================
# 가용재고 데이터
# =========================
path = 'W:/공용 드라이브/02. 재고/01. 재고현황 (일자별)/2025/'
target_files = []

t0 = time.time()
print('▶ 1️⃣ 가용재고 파일 수집 시작 (6월부터)')

# 1️⃣ 대상 파일 수집 (폴더 기준 MMDD, 6월부터)
for root, _, files in os.walk(path):
    date = os.path.basename(root)  # 예: '0601', '0715'

    # 🔹 날짜 폴더 검증 (MMDD)
    if not (len(date) == 4 and date.isdigit()):
        continue

    folder_month = int(date[:2])
    if folder_month < 6:
        continue  # 6월 이전 폴더 제외

    for f in files:
        if (
            '타점현재고현황_상세정보' in f and
            f.strip().replace(' ', '').lower().endswith('.xlsx')
        ):
            # 🔹 파일명에서 기준월 추출
            # 예시:
            # 타점현재고현황_상세정보_202505.xlsx
            # 타점현재고현황_상세정보_20250601.xlsx
            try:
                # 파일명에서 숫자 6~8자리 중 YYYYMM만 추출
                digits = ''.join(filter(str.isdigit, f))
                file_ym = digits[:6]        # YYYYMM
                file_month = int(file_ym[4:6])
            except Exception:
                continue

            if file_month < 6:
                continue  # ⭐ 6월 이전 기준월 제외

            full_path = os.path.join(root, f)
            target_files.append((full_path, date))

print(f'✔ 파일 수집 완료: {len(target_files)} files '
      f'({(time.time() - t0)/60:.1f}분 소요)')

if not target_files:
    raise ValueError('❌ 가용재고 파일이 조건에 맞게 하나도 없습니다')

# 2️⃣ for문 로딩
dfs = []

print('▶ 2️⃣ 파일 로딩 시작')

for i, (file_path, date) in enumerate(target_files, start=1):
    print(f'[{i}/{len(target_files)}] 로딩 중: {os.path.basename(file_path)}')

    df = pd.read_excel(file_path)

    # 🔹 컬럼명 정규화
    df.columns = (
        df.columns
        .astype(str)
        .str.strip()
        .str.replace(' ', '', regex=False)
    )

    # 🔹 기준일자 추가
    df['기준일자'] = date

    dfs.append(df)

print(f'✔ 파일 로딩 완료 '
      f'({(time.time() - t0)/60:.1f}분 경과)')

# 3️⃣ concat
t1 = time.time()
print('▶ 3️⃣ concat 시작')

stock_df = pd.concat(dfs, ignore_index=True)

print(f'✔ concat 완료 '
      f'({(time.time() - t1):.1f}초 소요)')

# 4️⃣ parquet 저장
t2 = time.time()
print('▶ 4️⃣ parquet 저장 시작')

save_path = r'C:/Users/user04/code/reserve_stock/reserve/'
os.makedirs(save_path, exist_ok=True)

stock_df.to_parquet(
    os.path.join(save_path, 'stock(2506_2512).parquet'),
    engine='pyarrow',
    index=False
)

print(f'✔ parquet 저장 완료 '
      f'({(time.time() - t2):.1f}초 소요)')

print('=' * 60)
print(f'✅ 전체 소요 시간: {(time.time() - t0)/60:.1f}분')
print('=' * 60)